In [24]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import load_model

from src.nn_settings.simple_gru_parameters import Encoder


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Save tf.keras model in HDF5 format.
keras_file = "../data/keras_rnn.h5"
tflite_file = "../data/converted_rnn.tflite"

test_data = np.array( [['nqXitzlidgYidtbfupgnzlxcf', 'itzlidg'],
                       ['li zo XnpdrYohtnpasa ', 'npdr'],
                       ['jkjnqxmmurpXrarrexwYafpvzplg', 'rarrexw'],
                       ['vpqmXmybvhYgnhrnbmipgipvzjygjyhgk', 'mybvh'],
                       ['qdzobuX wyYbskevnq  lrsonedurbgww', ' wy']])

char_string = "abcdefghijklmnopqrstuvwxyz XY"
characters = ["[STOP]", "[START]", "[UNK]"] + [ch for ch in char_string]
vocabulary_map = {}
for ii, ch in enumerate(characters):
    vocabulary_map[ch] = ii
enc = Encoder(vocabulary_map, max_input_length=43, max_output_length=7)
    
model = load_model(keras_file)

In [28]:
## Demonstrate that the model does something
test_data[:, 0]
test_enc = enc.encode(test_data[:, 0], encode_as="input")
pred = model.predict(test_enc)
enc.decode(pred, decode_as="output")

array([b'iiiiiii', b'rrrrrrr', b'rrrrrrr', b'mmmmmmm', b'yyyyyyy'],
      dtype='|S7')

In [29]:
tf.keras.models.save_model(model, keras_file)

## Convert to TensorFlow Lite model.
## These docs describe the converter in the TensorFlow nightly release, installed using pip install tf-nightly
# converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)

## For tensorflow 1.12 it should be 
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)


tflite_model = converter.convert()
open(tflite_file, "wb").write(tflite_model)

INFO:tensorflow:Froze 5 variables.
INFO:tensorflow:Converted 5 variables to const ops.


ValueError: None is only supported in the 1st dimension. Tensor 'gru_1_input' has invalid shape '[None, None, 32]'.

# Check performance of tflite model

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)